In [6]:
import os

from datasets import load_dataset
import pandas as pd

import requests
import json

from statistics import mean

import random
import csv
from nltk.corpus import gutenberg

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Presidential Speeches - Data

In [13]:
url = "https://millercenter.org/sites/default/files/corpus/presidential-speeches.json"
filename = "./data/Speeches/presidential-speeches.json"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the JSON data from the response
    json_data = response.json()

    # Save the JSON data to a file
    with open(filename, "w") as file:
        json.dump(json_data, file)

    print(f"JSON data saved to {filename} successfully.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


JSON data saved to ./data/Speeches/presidential-speeches.json successfully.


In [16]:
with open('./data/Speeches/presidential-speeches.json', 'r') as f:
  data = json.load(f)

speech_texts = [element["transcript"] for element in data]

for i in range(len(speech_texts)):
    speech_texts[i] = speech_texts[i].replace("\r\n\r\n", " ")
    speech_texts[i] = speech_texts[i].replace("\r\n", "")
  
speech_class = ['Political Speech'] * len(speech_texts)

df_speeches = pd.DataFrame({'Text': speech_texts, 'Class': speech_class})

In [17]:
print(speech_texts[0])

My countrymen, the surpassing war of all times has involved us, and found us utterly unprepared in either a mental or military sense. The Republic must awaken. The people must understand. Our safety lies in full realization the fate of the nation and the safety of the world will be decided on the western battlefront of Europe. Primarily the American Republic has entered the war in defense of its national rights. If we did not defend we could not hope to endure. Other big issues are involved but the maintained rights and defended honor of a righteous nation includes them all. Cherishing the national rights the fathers fought to establish, and loving freedom and civilization, we should have violated every tradition and sacrificed every inheritance if we had longer held aloof from the armed conflict which is to make the world safe for civilization. More, we are committed to sacrifice in battle in order to make America safe for Americans and establish their security on every lawful mission

In [18]:
array = []
for transcript in speech_texts:
    words = transcript.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

3929.8644230769232


# CNN News - Data

In [19]:
news_dataset = load_dataset("cnn_dailymail", "3.0.0")

Found cached dataset cnn_dailymail (/Users/bnnlukas/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
dataset_news = news_dataset['train']
news_texts = dataset_news['article'][:1000]

In [21]:
print(news_texts[5])

BAGHDAD, Iraq (CNN) -- Dressed in a Superman shirt, 5-year-old Youssif held his sister's hand Friday, seemingly unaware that millions of people across the world have been touched by his story. Nearby, his parents talked about the new future and hope they have for their boy -- and the potential for recovery from his severe burns. Youssif holds his sister's hand Friday. He's wearing a facial mask often used to help burn victims. It's the best birthday present the Iraqi family could ever have imagined for their boy: Youssif turns 6 next Friday. "I was so happy I didn't know what to do with myself," his mother, Zainab, told CNN, a broad smile across her face. "I didn't think the reaction would be this big." His father said he was on the roof of his house when CNN called him with the news about the outpouring of support for his son. "We just want to thank everyone who has come forward," he said. "We knew there was kindness out there." Like his wife, he couldn't stop smiling. He talked about

In [22]:
news_class = ['News'] * len(news_texts)

df_news = pd.DataFrame({'Text': news_texts, 'Class': news_class})

In [23]:
array = []
for text in news_texts:
    words = text.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

591.624


# Science - Data

In [43]:
science_dataset = load_dataset("scientific_papers", 'arxiv')

Found cached dataset scientific_papers (/Users/bnnlukas/.cache/huggingface/datasets/scientific_papers/arxiv/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_science = science_dataset['train']
science_texts = dataset_science['article'][:1000]

In [ ]:
science_texts = [w.replace('\n', ' ') for w in science_texts]

In [ ]:
science_texts[50]

'the study of the phase transition of frustrated spin systems on two - dimensional ( 2d ) lattices is a central problem in modern condensed mater physics . a competition of exchange interaction can lead to frustration , where spatial arrangement of magnetic ions in a crystal for which a simultaneous antiparallel ordering of all interacting spin is impossible . in particular , one of the frustrated 2d models most discussed is the quantum spin-@xmath0 heisenberg antiferromagnet on a square lattice with competing nearest - neighbor ( nn ) and next - nearest - neighbor ( nnn ) antiferromagnetic exchange interactions ( known as @xmath12 model ) @xcite . the criticality of this @xmath12 heisenberg model on a square lattice are relatively well known at @xmath13 . there are two magnetically long - range ordered phases at small and at large values of @xmath6 separated by an intermediate quantum paramagnetic phase without magnetic long - range order in the region between @xmath14 and @xmath15 , 

In [ ]:

# File path to write to
file_path = "./file.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
    # Write the string to the file
    file.write(science_texts[50])

In [ ]:
array = []
for text in science_texts:
    words = text.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

5793.014


# Jurisdictions - Data

In [24]:
data_path_jurisdictions = './data/Juristictions/'

jurisdictions_texts = []
for file_name in os.listdir(data_path_jurisdictions):
    file_path = os.path.join(data_path_jurisdictions, file_name)
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()
        jurisdictions_texts.append(text)

jurisdictions_class = ['Jurisdiction'] * len(jurisdictions_texts)

df_jurisdictions = pd.DataFrame({'Text': jurisdictions_texts, 'Class': jurisdictions_class})

array_jurisdictions = []
for text in jurisdictions_texts:
    words = text.split()
    num_words = len(words)
    array_jurisdictions.append(num_words)
print(mean(array_jurisdictions))

print(len(df_jurisdictions))

df_jurisdictions.head()


14266.597730138714
793


,Text,Class
0,"The appellant, VTB Capital plc (VTB), is incor...",Jurisdiction
1,"The appellants are insurance companies, whose ...",Jurisdiction
2,This appeal concerns the permissibility and in...,Jurisdiction
3,The Investigatory Powers Tribunal (IPT) is a s...,Jurisdiction
4,This appeal raises the issue as to whether a t...,Jurisdiction


# Literature - Data

In [25]:
anzahl_texte = 1000
ausgabe_datei = './data/Literature/literarische_texte.csv'
header = ['Titel', 'Text']

with open(ausgabe_datei, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    for i in range(anzahl_texte):
        buch = random.choice(gutenberg.fileids())
        titel = buch.replace('.txt', '')
        text = gutenberg.raw(buch)
        writer.writerow([titel, text])

In [9]:
df_lit = pd.read_csv('./data/Literature/literarische_texte.csv')

df_lit.head()

,Titel,Text
0,shakespeare-macbeth,[The Tragedie of Macbeth by William Shakespear...
1,whitman-leaves,[Leaves of Grass by Walt Whitman 1855]\n\n\nCo...
2,milton-paradise,[Paradise Lost by John Milton 1667] \n \n \nBo...
3,whitman-leaves,[Leaves of Grass by Walt Whitman 1855]\n\n\nCo...
4,edgeworth-parents,"[The Parent's Assistant, by Maria Edgeworth]\r..."


In [10]:
literature_texts = df_lit['Text']
literature_class = ['Literature'] * len(literature_texts)

df_literature = pd.DataFrame({'Text': literature_texts, 'Class': literature_class})

In [11]:
array = []
for transcript in df_lit['Text']:
    words = transcript.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

118295.018


# Result Dataframe

In [26]:
result_df = pd.concat([df_speeches, df_news, df_jurisdictions, df_literature])

result_df.head()

,Text,Class
0,"My countrymen, the surpassing war of all times...",Political Speech
1,"THE PRESIDENT. Good afternoon, ladies and gent...",Political Speech
2,"THE PRESIDENT. Ladies and gentlemen, I have co...",Political Speech
3,THE PRESIDENT. I take pleasure this morning in...,Political Speech
4,THE PRESIDENT: This past week the United State...,Political Speech
